# Classification Tests

spiegazione test

### Imports

In [1]:
import re
import xml.etree.ElementTree as ET
import nltk
from nltk.stem.snowball import ItalianStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import svm
import pandas as pd
import numpy as np
import collections
from sklearn.model_selection import cross_val_score
# Skopt functions
from sklearn.metrics import make_scorer
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold
from skopt import gp_minimize # Bayesian optimization using Gaussian Processes
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

In [2]:
#nltk.download()

## Classes

In [3]:
class CarWordsHandler:
    # https://github.com/n8barr/automotive-model-year-data
    def __init__(self, cars_file):
        self.brands_list = set()
        self.models_list = set()
        f = open(cars_file, "r")
        cars_list = f.read().splitlines()
        for i in range(len(cars_list)):
            brand = cars_list[i].split(',')[1][2:-1].lower()
            model = cars_list[i].split(',')[2][2:-2].lower()
            self.brands_list.add(brand)
            self.models_list.add(model)
        # remove some useless models
        self.models_list.remove('i')
        self.models_list.remove('gli')
        self.models_list.remove('estate')
        self.brands_list = list(self.brands_list)
        self.models_list = list(self.models_list)
        self.brands_list.sort()
        self.models_list.sort()
    
    # binary search to get if a word is a brand 
    def isBrand(self, word):
        word = word.lower()
        first = 0
        last = len(self.brands_list) -1
        while first < last:
            mid = int((last + first) / 2)
            if word == self.brands_list[mid]:
                return True
            elif word < self.brands_list[mid]:
                last = mid
            elif word > self.brands_list[mid]:
                first = mid
            if last-first == 1:
                if self.brands_list[first] == word or self.brands_list[last] == word:
                    return True
                else:
                    return False
    # binary search to get if a word is a brand 
    def isModel(self, word):
        word = word.lower()
        first = 0
        last = len(self.models_list) -1
        while first < last:
            mid = int((last + first) / 2)
            if word == self.models_list[mid]:
                return True
            elif word < self.models_list[mid]:
                last = mid
            elif word > self.models_list[mid]:
                first = mid
            if last-first == 1:
                if self.models_list[first] == word or self.models_list[last] == word:
                    return True
                else:
                    return False

## Functions

In [4]:
# encoding issues
def correctEncodings(comment):
    fin_comment = comment
    fin_comment = re.sub('Ã¨', 'è', fin_comment)
    fin_comment = re.sub('Ã©', 'é', fin_comment)
    fin_comment = re.sub('Ã¬', 'ì', fin_comment)
    fin_comment = re.sub('Ã²', 'ò', fin_comment)
    fin_comment = re.sub('Ã¹', 'ù', fin_comment)
    fin_comment = re.sub('Ã', 'à', fin_comment)
    return fin_comment
# recognize an URL
def isURL(word):
    # http://forum.rusconi.it/gentemotori/viewtopic.php ? t=434&sid=57c88f1b507d8f57717ea18e74e25324Â 
    return len(re.findall("^((http(s){0,1}://)|(www.))\S+$", word)) > 0
# recognize an image tag
def isPicture(word):
    return len(re.findall("^<img.*>$", word)) > 0
# remove punctation
def removePunctation(comment):
    return re.sub(r'\s{2,}', ' ', str(re.sub(r'[\'\"\,\.,\:\-]', ' ', comment)))
# fix issues on urls
def replaceURLs(comment):
    return str(re.sub(r'(http(s){0,1}://|www.)(([^\s]+)|/)+((\s\?\s)[^\s]+){0,1}', 'URL', comment)).replace(u'\xa0', u' ')
# replace images
def replaceIMGs(comment):
    return str(re.sub(r'<img.+>', 'IMG', comment))
# replace brands
def replaceBrands(cwhandler, comment):
    tokens = comment.split(' ')
    for i in range(len(tokens)):
        if cwhandler.isBrand(tokens[i]):
            tokens[i] = 'BRAND'
    return ' '.join(tokens)
# replace models
def replaceModels(cwhandler, comment):
    tokens = comment.split(' ')
    for i in range(len(tokens)):
        if cwhandler.isModel(tokens[i]):
            tokens[i] = 'MODEL'
    return ' '.join(tokens)
# replace question marks
def replaceQMarks(comment):
    comment = re.sub(r'\?{2,}', ' MULTI_QMARK', comment)
    comment = re.sub(r'\?', ' QMARK', comment)
    return comment
# replace esclamation marks
def replaceEMarks(comment):
    comment = re.sub(r'\!{2,}', ' MULTI_EMARK', comment)
    comment = re.sub(r'\!', ' EMARK', comment)
    return comment
# remove character repetitions
def removeRepeat(comment):
    return re.sub(r'(a-zA-Z)\1{2,}', r'\1\1\1', comment)
# replace speed
def replaceSpeed(comment):
    return re.sub(r'([0-9\.*]+(\s*(\-|\/|\s)\s*)+){0,1}[0-9\.*]+(\s*)(km\/h|mph)', 'SPEED', comment)
# replace consumption
def replaceConsumption(comment):
    return re.sub(r'([0-9\.*]+(\s*(\-|\/|\s)\s*)+){0,1}[0-9\.*]+(\s*)(km\/l|mpg)', 'CONSUMPTION', comment)
# replace weight
def replaceWeight(comment):
    return re.sub(r'[0-9\.*]+(\s*)(kg|tonnellate|ton|chili|kili)', 'WEIGHT', comment)

In [5]:
class ItalianWordsHandler:
    # https://dspace-clarin-it.ilc.cnr.it/repository/xmlui/handle/20.500.11752/ILC-73
    def __init__(self, words_file):
        # words information
        self.words_dict = dict()
        root = ET.parse(words_file).getroot()
        for entry in root.findall('Lexicon/LexicalEntry'):
            word = entry.find('Lemma').get('writtenForm')
            pos = entry.get('partOfSpeech')
            senti = entry.find('Sense/Sentiment').get('polarity')
            conf = entry.find('Sense/Confidence').get('score')
            self.words_dict[word] = {'POS': pos, 'Sentiment': senti, 'Confidence': conf}
        # stemmer
        self.it_stem = ItalianStemmer()
        
    # get word info. None if not exists
    def getWordInfo(self, word):
        # fields: POS, Sentiment, Confidence
        return self.words_dict.get(word)
    
    # italian stemmer http://snowball.tartarus.org/algorithms/italian/stemmer.html
    def stem(self, word):
        return self.it_stem.stem(word)
    
    # correct words
    def correctWords(self, text):
        # not yet implemented
        return text

In [6]:
class Preprocessor:
    
    def __init__(self):
        self.cwh = CarWordsHandler('resources/cars_data.sql')
        self.iwh = ItalianWordsHandler('resources/ita_opeNER.xml')
    # preprocess text
    # allowed methods: word, swnt, pos
    # ner (named entity recognition), replacing for instance 100 km/h with SPEED
    def preprocessText(self, text, method='word', use_stemmer=False, ner=False):
        if method not in ['word', 'pos', 'swnt']:
            raise ValueError('Method not recognized. Select from word, swnt, pos')
        # correct encodings (not yet implemented)
        fin_text = correctEncodings(text)
        # remove punctation
        fin_text = removePunctation(fin_text)
        # some basic preprocessing
        fin_text = fin_text.lower()
        # correct words (not yet)
        fin_text = self.iwh.correctWords(fin_text)
        # manage repetitions
        fin_text = removeRepeat(fin_text)
        # manage punctation
        fin_text = replaceQMarks(fin_text)
        fin_text = replaceEMarks(fin_text)
        # manage URLs
        fin_text = replaceURLs(fin_text)
        # manage Images
        fin_text = replaceIMGs(fin_text)
        # NOW DEPENDS ON NER
        if ner:
            # manage brands and models
            fin_text = replaceBrands(self.cwh, fin_text)
            fin_text = replaceModels(self.cwh, fin_text)
            # manage speed consumption and weight
            fin_text = replaceSpeed(fin_text)
            fin_text = replaceConsumption(fin_text)
            fin_text = replaceWeight(fin_text)
        # NOW DEPENDS ON METHOD
        if method == 'word':
            # just do nothing except eventually stemming
            if use_stemmer:
                tokens = fin_text.split(' ')
                fin_text = ' '.join([t if t.isupper() else self.iwh.stem(t) for t in tokens]) 
        elif method == 'swnt':
            tokens = fin_text.split(' ')
            swnt_tokens = []
            for t in tokens:
                info = self.iwh.getWordInfo(t)
                if info == None or info['Sentiment'] == None:
                    swnt_tokens.append(t)
                else:
                    # confidence 0-100
                    swnt_tokens.append(str(info['Sentiment'])[:3].upper() + '_' + str(int(float(info['Confidence'])*10)))
            fin_text = ' '.join(swnt_tokens)
            # stemmer
            if use_stemmer:
                tokens = fin_text.split(' ')
                fin_text = ' '.join([t if t.isupper() else self.iwh.stem(t) for t in tokens]) 
        elif method == 'pos':
            tokens = fin_text.split(' ')
            pos_tokens = []
            # pos
            for t in tokens:
                info = self.iwh.getWordInfo(t)
                if info == None or info['POS'] == None:
                    # unknown tag
                    pos_tokens.append('UNK')
                else:
                    pos_tokens.append(str(info['POS']).upper())
            # pos_word
            for t in tokens:
                info = self.iwh.getWordInfo(t)
                if info == None or info['POS'] == None:
                    pos_tokens.append('UNK_' + str(t))
                else:
                    pos_tokens.append(str(info['POS']).upper() + '_' + str(t))
                    
            fin_text = ' '.join(pos_tokens)
            # stemmer
            if use_stemmer:
                tokens = fin_text.split(' ')
                fin_text = ' '.join([t if t.isupper() else self.iwh.stem(t) for t in tokens]) 
        
        return str(re.sub(r'\s{2,}', ' ', fin_text))

In [7]:
class Vectorizer:
    
    def __init__(self, list_comments, method='bow', max_features=1000, ngrams=2, just_presence=False):
        if method not in ['bow', 'tfidf']:
            raise ValueError('Method not recognized. Select from bow, tfidf')
        if method == 'bow':
            self.vectorizer = CountVectorizer(ngram_range=(1,ngrams), binary=just_presence, lowercase=False, max_features=max_features)
        elif method == 'tfidf':
            self.vectorizer = TfidfVectorizer(ngram_range=(1,ngrams), lowercase=False, max_features=max_features)
        # fit vectorizer
        self.vectorizer.fit(list_comments)          
        
        
        '''
        self.list_comments = list_comments
        # initialize tfidf weights
        self.idf_dict = {}
        idf_dict_uni = {}
        idf_dict_big = {}
        # unigrams
        for comment in list_comments:
            tokens = list(set(comment.split()))
            for t in tokens:
                if idf_dict_uni.get(t) != None:
                    idf_dict_uni[t] = idf_dict_uni[t] + 1
                else:
                    idf_dict_uni[t] = 1
        # bigrams
        for comment in list_comments:
            tokens = comment.split()
            for i in range(len(tokens) -2):
                big = (tokens[i], tokens[i+1])
                if idf_dict_big.get(big) != None:
                    idf_dict_big[big] = idf_dict_big[big] + 1
                else:
                    idf_dict_big[big] = 1
        # cut most frequent
        idf_dict_uni = Counter(idf_dict_uni).most_common(most_common_unigrams)
        idf_dict_big = Counter(idf_dict_big).most_common(most_common_bigrams)
        self.idf_dict.update(idf_dict_uni)
        self.idf_dict.update(idf_dict_big)
        '''
        
    def vectorize(self, comment):
        
        return self.vectorizer.transform([comment])
        
        '''
        if method not in ['bow', 'tfidf']:
            raise ValueError('Method not recognized. Select from bow, tfidf')
        unigrams = comment.split(' ')
        bigrams = []
        for i in range(len(unigrams) -2):
            bigrams.append((unigrams[i], unigrams[i+1]))
        if method == 'bow':
            bow_dict = dict.fromkeys(self.idf_dict, 0)
            for u in unigrams + bigrams:
                if bow_dict.get(u) != None:
                    if just_presence:
                        bow_dict[u] = 1
                    else:
                        bow_dict[u] = bow_dict[u] +1
            return list(bow_dict.values())
        elif method == 'tfidf':
            tf_dict = dict.fromkeys(self.idf_dict, 0)
            for u in unigrams + bigrams:
                if tf_dict.get(u) != None:
                    tf_dict[u] = tf_dict[u] +1
        '''
        
    def get_feature_names(self):
        return self.vectorizer.get_feature_names()

Vectorization Example

In [8]:
v = Vectorizer(list_comments=['ciao come va ?', 'vediamo come scrive ciao questa tastiera', 'non lo so, per me è falso'], method='tfidf', max_features=100, ngrams=2, just_presence=True)
print(v.get_feature_names())
print(v.vectorize('ciao ciao come va ?'))

['ciao', 'ciao come', 'ciao questa', 'come', 'come scrive', 'come va', 'falso', 'lo', 'lo so', 'me', 'me falso', 'non', 'non lo', 'per', 'per me', 'questa', 'questa tastiera', 'scrive', 'scrive ciao', 'so', 'so per', 'tastiera', 'va', 'vediamo', 'vediamo come']
  (0, 22)	0.41197297843389025
  (0, 5)	0.41197297843389025
  (0, 3)	0.3133160688892059
  (0, 1)	0.41197297843389025
  (0, 0)	0.6266321377784118


Preprocessing Example

In [9]:
text = 'Sono reali calcolati nel arco del tutto anno nel estate qualcosa in piÃ¹ causa gomme di 17" e climatizzatore nel inverno un po di meno. Per quanto riguarda le autostrade quelle che percorro io principalmente la A4 e molto congestionata cosi spesso la media e 110-115 km/h che ovviamente influisce positivamente a i consumi. Ma quello che mi piace di piÃ¹ Ã¨ assenza dei guasti. Sulla vecchia Accord il primo guasto lo ho avuto a 200000 km si Ã¨ rotto il termostato della clima. Ogni tanto faccio giro di altri forum e leggo delle turbine rotte catene di distribuzione progettate male iniettori fatti male mah nel 2015 per me sono le cose incomprensibili . Con tutti gli difetti che puÃ² avere preferisco la Honda. '
print(text)
print('##########################################################################################')
p = Preprocessor()
print(p.preprocessText(text, ner=False, use_stemmer=False, method='word'))
print('##########################################################################################')
print(p.preprocessText(text, ner=False, use_stemmer=False, method='swnt'))
print('##########################################################################################')
print(p.preprocessText(text, ner=False, use_stemmer=False, method='pos'))


Sono reali calcolati nel arco del tutto anno nel estate qualcosa in piÃ¹ causa gomme di 17" e climatizzatore nel inverno un po di meno. Per quanto riguarda le autostrade quelle che percorro io principalmente la A4 e molto congestionata cosi spesso la media e 110-115 km/h che ovviamente influisce positivamente a i consumi. Ma quello che mi piace di piÃ¹ Ã¨ assenza dei guasti. Sulla vecchia Accord il primo guasto lo ho avuto a 200000 km si Ã¨ rotto il termostato della clima. Ogni tanto faccio giro di altri forum e leggo delle turbine rotte catene di distribuzione progettate male iniettori fatti male mah nel 2015 per me sono le cose incomprensibili . Con tutti gli difetti che puÃ² avere preferisco la Honda. 
##########################################################################################
sono reali calcolati nel arco del tutto anno nel estate qualcosa in più causa gomme di 17 e climatizzatore nel inverno un po di meno per quanto riguarda le autostrade quelle che percorro io prin

For each comment divide Text from Quote by setting _TEXT or _QUOTE at the end of each word. This after preprocessing

In [10]:
def combine_text_quote(text, quote):
    text_tokens = text.split(' ')
    quote_tokens = quote.split(' ')
    combined_tokens = []
    for tt in text_tokens:
        combined_tokens.append(str(tt) + '_TEXT')
    for qt in quote_tokens:
        combined_tokens.append(str(qt) + '_QUOTE')
    return ' '.join(combined_tokens)

In [11]:
print(combine_text_quote('Bene grazie', 'ciao come va'))

Bene_TEXT grazie_TEXT ciao_QUOTE come_QUOTE va_QUOTE


# Dataset

In [12]:
dataset = pd.read_csv('dataset.csv').fillna('')[['TESTO', '(Testo Citato)', 'Brand']]
dataset

,TESTO,(Testo Citato),Brand
0,Allora il problema è che non sono aggiornati i...,,irrilevante
1,E' virgolettato appositamente.... E soprattutt...,,irrilevante
2,Mah io sulla mappa ev-way ho visto solo tipo 2...,'Inferiore non s ma probabilmente uguale (il m...,irrilevante
3,Ah però.... uno pensa di averne viste tante su...,Sinceramente una differenza di 9.800? non mi p...,irrilevante
4,Basta darsi delle regole e per questo tipo di ...,,irrilevante
5,Personalmente posso ritenermi un possessore di...,,positivo
6,Confermo che il posteriore è riuscitissimo e a...,"""sono stato anch'io in conce per altri motivi ...",irrilevante
7,Mi sa che l'unica cosa apprezzabile del rst è ...,,irrilevante
8,grazie ! ! ! ! !,,irrilevante
9,Cosimo hai centrato completamente,,irrilevante


Preprocessing

In [13]:
pp = Preprocessor()
# TESTO
for i in range(len(dataset)):
    dataset['TESTO'][i] = pp.preprocessText(dataset['TESTO'][i], method='word', use_stemmer=True, ner=True)
# (Testo Citato)
for i in range(len(dataset)):
    dataset['(Testo Citato)'][i] = pp.preprocessText(dataset['(Testo Citato)'][i], method='word', use_stemmer=True, ner=True)    

In [14]:
# combine labels "molto positivo" = "positivo" and so on
dataset['Brand'] = dataset['Brand'].replace('molto positivo', 'positivo')
dataset['Brand'] = dataset['Brand'].replace('molto negativo', 'negativo')

Vectorization

In [15]:
preprocessed_dataset = []
for i in range(len(dataset)):
    preprocessed_dataset.append((combine_text_quote(dataset['TESTO'][i], dataset['(Testo Citato)'][i]), dataset['Brand'][i]))

In [16]:
preprocessed_dataset[10]

('bisogn_TEXT distingu_TEXT tra_TEXT le_TEXT garanz_TEXT supplementar_TEXT cio_TEXT quell_TEXT acquist_TEXT success_TEXT in_TEXT cui_TEXT firm_TEXT un_TEXT ver_TEXT e_TEXT pror_TEXT contratt_TEXT un_TEXT cui_TEXT esul_TEXT l_TEXT applic_TEXT del_TEXT decret_TEXT mont_TEXT e_TEXT la_TEXT garanz_TEXT tip_TEXT quell_TEXT corean_TEXT che_TEXT son_TEXT dirett_TEXT di_TEXT 5/7_TEXT anni_TEXT EMARK_TEXT _TEXT _QUOTE',
 'irrilevante')

Split into training and test

In [17]:
numpy_dataset = np.array(preprocessed_dataset)
print('Length: ' + str(numpy_dataset.shape[0]))

Length: 7183


In [18]:
np.random.seed(26)
np.random.shuffle(numpy_dataset)
train_dataset, test_dataset = numpy_dataset[:6000,:], numpy_dataset[6000:,:]

Some statistics

In [19]:
collections.Counter(train_dataset[:,1])

Counter({'irrilevante': 4867, 'negativo': 349, 'positivo': 543, 'neutro': 241})

In [20]:
# fit vectorizer
MAX_FEATURES = 10000
vec = Vectorizer(train_dataset[:,0], method='bow', max_features=MAX_FEATURES, ngrams=2, just_presence=False)

In [21]:
# try
vec.vectorize('bisogn_TEXT distingu_TEXT tra_TEXT le_TEXT').toarray()[0]

array([0, 0, 0, ..., 0, 0, 0])

Finally replace text with vector, and replace label with numerical representation

In [22]:
# final dataset for the classification
train_X = np.empty([train_dataset.shape[0], MAX_FEATURES])
train_y = np.empty([train_dataset.shape[0], 1])

for i in range(train_dataset.shape[0]):
    #train_X[i] = vec.vectorize(train_dataset[i, 0]).toarray()[0].copy()
    # copy vectorized elements in final dataset
    vectorized = vec.vectorize(train_dataset[i, 0]).toarray()[0]
    for j in range(train_X.shape[1]):
        train_X[i,j] = vectorized[j]
    if train_dataset[i, 1] == 'irrilevante':
        train_y[i] = 0
    elif train_dataset[i, 1] == 'positivo':
        train_y[i] = 1
    elif train_dataset[i, 1] == 'neutro':
        train_y[i] = 2
    elif train_dataset[i, 1] == 'negativo':
        train_y[i] = 3

## SVM _TFIDF Classifier

https://towardsdatascience.com/automated-machine-learning-hyperparameter-tuning-in-python-dfda59b72f8a

In [23]:
svm_clf = svm.SVC()

# Setting a 5-fold stratified cross-validation (note: shuffle=True)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
# scorer
my_scorer = make_scorer(f1_score)

# Define search space for SVM;
search_space_SVM = {"C": Real(0.01, 10),
                    "kernel": Categorical(categories=['linear', 'rbf', 'poly', 'sigmoid']),
                    "degree": Categorical(categories=[3, 5]),
                    "gamma": Categorical(categories=['scale']),
                    "coef0": Real(0.01, 10),
                    "class_weight": Categorical(categories=['balanced']),
                    "probability": Categorical(categories=[True, False]),
                    "decision_function_shape": Categorical(categories=['ovo', 'ovr'])
                   }

opt = BayesSearchCV(svm_clf,
                    search_spaces=search_space_SVM,
                    scoring=my_scorer,
                    n_iter=40,
                    cv=skf,
                    n_jobs=-1,
                    return_train_score=True,
                    random_state=4,
                    verbose=10,
                   )

In [24]:
# fit model
opt.fit(train_X, train_y)

best_score = opt.best_score_
best_score_std = opt.cv_results_['std_test_score'][opt.best_index_]
best_params = opt.best_params_

print('Best score: ' + str(best_score) + " std: " + str(best_score_std))
print('Best params: ' + str(best_params))
#preds = opt.predict(x_test)
#print('f1-score: ' + str(f1_score(y_test, preds)))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] C=8.216234727026476, class_weight=balanced, coef0=2.6829639217572407, decision_function_shape=ovr, degree=3, gamma=scale, kernel=rbf, probability=False 
[CV] C=8.216234727026476, class_weight=balanced, coef0=2.6829639217572407, decision_function_shape=ovr, degree=3, gamma=scale, kernel=rbf, probability=False 


/home/giuseppe/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


JoblibTypeError: JoblibTypeError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/giuseppe/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/home/giuseppe/anaconda3/lib/

In [25]:
import sklearn
print(sklearn.__version__)

0.19.1
